## Chlorophyll-a forecasting using LSTM Model



#### Getting Started:
1. Before running the notebook, please make sure to have the following python version and libraries are installed <br>
- python 3.9.12
- pytorch (https://pytorch.org/get-started/locally/)

2. Create an account in Weights and Biases (WANDB) (https://wandb.ai/home). While running the notebook, you maybe prompted to enter the WANDB username

<br>
The requirements.txt file lists the basic libraries require. Running the following cell should install all of them (in case they are not already installed). 

In case, any library is missed here, you would be prompted with an ImportError. In such case, just install it with pip (google -> pip install library_name)

In [35]:
!pip install -r requirements.txt

In [36]:
import random
import pandas as pd
import numpy as np
from tqdm import trange
import os
import datetime
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch import optim

from utils import Utils
from encoder_decoder import seq2seq

import warnings
warnings.filterwarnings('ignore')

## 0. GPU Selection
Check if GPU is available on the machine the notebook is running. If yes, then assign a GPU, else run it on CPU

In [37]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print(device)

cuda


## 1. Parameter setting

#### Specify the wandb project and wandb run
wandb refers to Weights and Biases. Integrating this tool into the notebook will allow it to access the run details and generate train and test curves, among many other information

In [38]:
# wandb project name
wandb_project = "lake_modeling"

# wandb run name
wandb_run = "test_run_{}_{}".format(str(datetime.datetime.now().date()), str(datetime.datetime.now().time()))

# Yes if we want wandb to save our python code, else no
save_code = True

#### Specify the input path (where the dataset is stored) and dataset name
Note: For different dataset, the processing/handling can/will be different. In this notebook, FCR (observational) data has been considered. It also has a metadata file that stores the column names and types. 
<br>
For the purpose of the tutorial, the notebook is kept simple, hence, going with FCR data for now

In [39]:
# Input path
path = './'

# Name of the file
file = '../1_trainingData/all_data_lake_modeling_in_time_perRow.csv'

# Name of the metadata file
#../metadata = 'LSTM_dataset_column_key_07OCT22.csv'

#### Specify the Time-series specific parameters

In [40]:
# Lookback window
input_window = 24*7

# horizon window
output_window = 24*7

# stride - While creating samples (lookback window + horizon window = 1 sample) define the amount of stride the sliding window needs to take
stride = 1

# The ratio in which train and test data is split. If it is 0.8, then first 80% of data goes into train and remaining 20% into test
split_ratio = 0.6

#### Specify the model specific parameters

In [41]:
# Types of Model include: LSTM, GRU, RNN
model_type = "LSTM"

# Number of layers in our deep learning model
num_layers = 1

# Hidden cell (RNN/LSTM/GRU) size
hidden_feature_size = 16

# Output size of our encoder_decoder model, i.e. number of target variables
output_size = 50

'''
Model Training parameters
'''
# batch_size during training
batch_size = 32

# Number of epochs we want to train the model for (1 epoch = 1 pass of the complete training data through the model)
epochs = 200

# Learning rate specifies the rate at which we want to update the model parameters after every training pass
learning_rate = 0.001

# Eval freq says how frequently during training do you want to evaluate your model on the validation data (to see its performance on non-training data)
eval_freq = 10 # logic is -> if iteration_num % eval_freq == 0 -> then perform evaluation

# While generating the training batches do we want the generator to shuffle the batches?
batch_shuffle = True

# Dropout is a form of regularization
dropout = 0.02

'''
Learning rate scheduler parameters
'''
max_lr=5e-4
div_factor=100
pct_start=0.05 
anneal_strategy='cos'
final_div_factor=10000.0

'''
Parameters for early stopping
'''
# Set to True if we want Early stopping
early_stop = False

# If there is no improvement for a 'thres' number of epocs stop the training process
thres=5

# Quantifying the improvement. If the validation loss is greater than min_val_loss_so_far + delta for thres number of iterations stop the training
delta=0.5

'''
Other parameters
'''
# Specify the amount of L2 regularization to be applied.
weight_decay=0.05

# Specify the percentage of times we want to enforce teacher forcing
teacher_forcing_ratio = 0.0
training_prediction = 'recursive'

## 2. Data Processing

#### Read the metadata file

In [42]:
depth_steps = 25 * 2 

depth_list = np.array(list(range(1, depth_steps+1))   )*0.5



In [43]:
incoming_temp = ['temp_initial00_{}'.format(x) for x in depth_list]
outgoing_temp = ['temp_heat01_{}'.format(x) for x in depth_list]

dx = pd.read_csv(os.path.join(path,file))

#feature_cols = ['AirTemp_degC', 'Longwave_Wm-2', 'Latent_Wm-2', 'Sensible_Wm-2', 'Shortwave_Wm-2',
#                'lightExtinct_m-1', 'ShearStress_Nm-2',
#                 'day_of_year', 'time_of_day', 'ice', 'snow', 'snowice','Volume_m2','Osgood','MaxDepth_m',
#                'MeanDepth_m','Area_m2'] + incoming_temp

feature_cols = ['AirTemp_degC', 'Longwave_Wm-2', 'Latent_Wm-2', 'Sensible_Wm-2', 'Shortwave_Wm-2',
                'lightExtinct_m-1', 'ShearStress_Nm-2',
                 'day_of_year', 'time_of_day', 'ice', 'snow', 'snowice'] + incoming_temp

date_col = ['time']

target_cols = outgoing_temp

In [44]:
#dx = pd.read_csv(os.path.join(path, metadata))

# Extract all col names from Metadata
#feature_cols = dx[dx['column_type']=='feature']['column_names'].tolist()  # feature colums represent the input drivers
#target_cols = dx[dx['column_type']=='target']['column_names'].tolist()   # target column represent the chlorophyll values
#date_col = dx[dx['column_type']=='date']['column_names'].tolist()[0]    # date column stores the date timeline

In [45]:
# Specify whether we want to add chlorophyll to the input feature set
#feature_cols += target_cols
feature_cols

['AirTemp_degC',
 'Longwave_Wm-2',
 'Latent_Wm-2',
 'Sensible_Wm-2',
 'Shortwave_Wm-2',
 'lightExtinct_m-1',
 'ShearStress_Nm-2',
 'day_of_year',
 'time_of_day',
 'ice',
 'snow',
 'snowice',
 'temp_initial00_0.5',
 'temp_initial00_1.0',
 'temp_initial00_1.5',
 'temp_initial00_2.0',
 'temp_initial00_2.5',
 'temp_initial00_3.0',
 'temp_initial00_3.5',
 'temp_initial00_4.0',
 'temp_initial00_4.5',
 'temp_initial00_5.0',
 'temp_initial00_5.5',
 'temp_initial00_6.0',
 'temp_initial00_6.5',
 'temp_initial00_7.0',
 'temp_initial00_7.5',
 'temp_initial00_8.0',
 'temp_initial00_8.5',
 'temp_initial00_9.0',
 'temp_initial00_9.5',
 'temp_initial00_10.0',
 'temp_initial00_10.5',
 'temp_initial00_11.0',
 'temp_initial00_11.5',
 'temp_initial00_12.0',
 'temp_initial00_12.5',
 'temp_initial00_13.0',
 'temp_initial00_13.5',
 'temp_initial00_14.0',
 'temp_initial00_14.5',
 'temp_initial00_15.0',
 'temp_initial00_15.5',
 'temp_initial00_16.0',
 'temp_initial00_16.5',
 'temp_initial00_17.0',
 'temp_initi

In [46]:
target_cols

['temp_heat01_0.5',
 'temp_heat01_1.0',
 'temp_heat01_1.5',
 'temp_heat01_2.0',
 'temp_heat01_2.5',
 'temp_heat01_3.0',
 'temp_heat01_3.5',
 'temp_heat01_4.0',
 'temp_heat01_4.5',
 'temp_heat01_5.0',
 'temp_heat01_5.5',
 'temp_heat01_6.0',
 'temp_heat01_6.5',
 'temp_heat01_7.0',
 'temp_heat01_7.5',
 'temp_heat01_8.0',
 'temp_heat01_8.5',
 'temp_heat01_9.0',
 'temp_heat01_9.5',
 'temp_heat01_10.0',
 'temp_heat01_10.5',
 'temp_heat01_11.0',
 'temp_heat01_11.5',
 'temp_heat01_12.0',
 'temp_heat01_12.5',
 'temp_heat01_13.0',
 'temp_heat01_13.5',
 'temp_heat01_14.0',
 'temp_heat01_14.5',
 'temp_heat01_15.0',
 'temp_heat01_15.5',
 'temp_heat01_16.0',
 'temp_heat01_16.5',
 'temp_heat01_17.0',
 'temp_heat01_17.5',
 'temp_heat01_18.0',
 'temp_heat01_18.5',
 'temp_heat01_19.0',
 'temp_heat01_19.5',
 'temp_heat01_20.0',
 'temp_heat01_20.5',
 'temp_heat01_21.0',
 'temp_heat01_21.5',
 'temp_heat01_22.0',
 'temp_heat01_22.5',
 'temp_heat01_23.0',
 'temp_heat01_23.5',
 'temp_heat01_24.0',
 'temp_heat

#### Create an utility object
An object of the Utils class, it contains all the utility functions like splitting train and test data, normalizing the data, etc.

In [47]:
'''
Utility instance - to perform data processing, train test split
'''
utils = Utils(num_features=len(feature_cols), inp_cols=feature_cols, target_cols=target_cols, date_col=date_col,
              input_window=input_window, output_window=output_window, num_out_features=output_size, stride=stride)

#### Read the dataset

In [48]:
'''
Read data
'''
df = pd.read_csv(path+file)

#### Train Test split
Ideally, a 3-way split is done - train, val and test. The validation split is generally used to tune the hyper-parameters during training. Once the hyper-parameters are tuned, the model
is re-trained on the train+val data. To keep the notebook short and simple, hyper-parameter tuning is not included

In [49]:
'''
Split data into train and test
'''
df_train, df_test = utils.train_test_split(df, split_ratio=split_ratio)

In [50]:
df_train

,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,TKE_Jm-1,ShearStress_Nm-2,Area_m2,...,input_obs_temp22.0,input_obs_temp22.5,input_obs_temp23.0,input_obs_temp23.5,input_obs_temp24.0,input_obs_temp24.5,input_obs_temp25.0,ice,snow,snowice
0,2012-05-19 01:00:00,12.800326,-59.975532,-72.115828,-16.270492,0.000000,0.6,1.903681e+07,0.026008,39850000.0,...,10.581667,10.581667,10.581667,10.581667,10.581667,10.513542,10.594643,0.000000,0.0,0.0
1,2012-05-19 02:00:00,12.140192,-56.638577,-70.510837,-18.766448,0.000000,0.6,1.770408e+07,0.024780,39850000.0,...,10.583333,10.583333,10.583333,10.583333,10.583333,10.514583,10.596429,0.000000,0.0,0.0
2,2012-05-19 03:00:00,11.480188,-53.244302,-70.044739,-21.813652,0.000000,0.6,1.649237e+07,0.023636,39850000.0,...,10.585000,10.585000,10.585000,10.585000,10.585000,10.515625,10.598214,0.000000,0.0,0.0
3,2012-05-19 04:00:00,10.820185,-49.665116,-70.307884,-25.217532,0.000000,0.6,1.540771e+07,0.022588,39850000.0,...,10.586667,10.586667,10.586667,10.586667,10.586667,10.516667,10.600000,0.000000,0.0,0.0
4,2012-05-19 05:00:00,11.689755,-35.756041,-63.667974,-18.685219,0.000000,0.6,1.631201e+07,0.023464,39850000.0,...,10.588333,10.588333,10.588333,10.588333,10.588333,10.517708,10.601786,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31537,2015-12-23 17:00:00,12.390125,40.856598,104.914893,119.311101,30.364047,0.6,0.000000e+00,0.098478,39850000.0,...,6.066909,6.026453,6.126453,6.105087,6.033721,6.083721,9.625890,0.144374,0.0,0.0
31538,2015-12-23 18:00:00,11.980149,38.626374,96.194488,109.318901,0.008449,0.6,0.000000e+00,0.090045,39850000.0,...,6.061919,6.021512,6.121512,6.100291,6.029070,6.079070,9.625959,0.141313,0.0,0.0
31539,2015-12-23 19:00:00,11.560108,36.264525,88.203586,100.056888,0.000000,0.6,0.000000e+00,0.082538,39850000.0,...,6.056928,6.016570,6.116570,6.095494,6.024419,6.074419,9.626027,0.138503,0.0,0.0
31540,2015-12-23 20:00:00,11.010156,32.104582,78.434475,92.457032,0.000000,0.6,0.000000e+00,0.077731,39850000.0,...,6.051938,6.011628,6.111628,6.090698,6.019767,6.069767,9.626096,0.135963,0.0,0.0


#### Normalize the data
Standard normalization - 0 mean and 1 standard deviation

In [51]:
'''
Data Scaling
'''
df_train = utils.normalize(df_train)

In [ ]:
df_train.head

In [52]:
df_test = utils.normalize(df_test, use_stat=True)

In [53]:
df_test.head

<bound method NDFrame.head of                       time  AirTemp_degC  Longwave_Wm-2  Latent_Wm-2  \
0      2015-12-23 22:00:00      0.041303       1.631190     2.624505   
1      2015-12-23 23:00:00     -0.121956       1.282957     2.167548   
2      2015-12-24 00:00:00     -0.285991       0.923642     1.694477   
3      2015-12-24 01:00:00     -0.450033       0.550149     1.034302   
4      2015-12-24 02:00:00     -0.531329       0.407681     0.822045   
...                    ...           ...            ...          ...   
21023  2018-05-17 18:00:00      0.888353       0.578021     0.722357   
21024  2018-05-17 19:00:00      0.799623       0.500690     0.694285   
21025  2018-05-17 20:00:00      0.640823       0.294908     0.509183   
21026  2018-05-17 21:00:00      0.482004       0.098219     0.327400   
21027  2018-05-17 22:00:00      0.323184      -0.088448     0.146148   

       Sensible_Wm-2  Shortwave_Wm-2  lightExtinct_m-1      TKE_Jm-1  \
0           1.980200       -0.706

#### Create train and test samples
Each sample is created using a sliding window. 1 sliding window = 1 lookback window + 1 horizon window = 1 sample

In [54]:
'''
Prepare data : 1 training sample = lookback window + horizon window
'''
Xtrain, Ytrain = utils.windowed_dataset(df_train)
Xtest, Ytest = utils.windowed_dataset(df_test)

#### Datatype conversion to torch

In [55]:
'''
Convert data into torch type
'''
X_train, Y_train, X_test, Y_test = utils.numpy_to_torch(Xtrain, Ytrain, Xtest, Ytest)

In [56]:
X_train.shape

torch.Size([31207, 168, 62])

## 3. Modeling

#### Define the model

In [57]:
'''
Create the seq2seq model
'''
model = seq2seq(input_size = X_train.shape[2], 
                hidden_size = hidden_feature_size, 
                output_size=output_size,
                model_type=model_type,
                num_layers = num_layers,
                utils=utils,
                dropout=dropout,
                device=device
               )

#### Train the model

In [ ]:
'''
Train the model
'''
config = {
    "batch_size": batch_size,
    "epochs": epochs,
    "learning_rate": learning_rate,
    "eval_freq": eval_freq,
    "batch_shuffle": batch_shuffle,
    "dropout":dropout,
    "num_layers": num_layers,
    "hidden_feature_size": hidden_feature_size,
    "model_type": model_type,
    "teacher_forcing_ratio": teacher_forcing_ratio,
    "max_lr": max_lr,
    "div_factor": div_factor,
    "pct_start": pct_start,
    "anneal_strategy": anneal_strategy,
    "final_div_factor": final_div_factor,
    "dataset": file,
    "split_ratio":split_ratio,
    "input_window":input_window,
    "output_window":output_window,
    "early_stop_thres":thres,
    "early_stop_delta":delta,
    "early_stop":early_stop,
    "weight_decay":weight_decay
}
loss, test_rmse, train_rmse = model.train_model(X_train, 
                                                Y_train,
                                                X_test,
                                                Y_test,
                                                target_len = output_window,
                                                config = config,
                                                training_prediction = training_prediction,  
                                                dynamic_tf = False,
                                                project_name = wandb_project,
                                                run_name = wandb_run,
                                                save_code = save_code)

 61%|██████████████████▎           | 122/200 [5:10:18<3:18:58, 153.06s/it, loss=0.527, test_rmse=7.13, train_rmse=6.79]

#### Plot the train test curves

In [ ]:
plt.figure(figsize=(5,4), dpi=150)
plt.plot(train_rmse, lw=2.0, label='train_rmse')
plt.plot(test_rmse, lw=2.0, label='test_rmse')
plt.yscale("log")
plt.grid("on", alpha=0.2)
plt.legend()
plt.show()

#### Save the model

In [ ]:
load = False

# If load=True, specify the model to load in the below line
MODEL_PATH = "./models/model_weights_test_run_2023-03-21_22:42:45.577525"

In [ ]:
if load:
    model.load_state_dict(torch.load(MODEL_PATH))
else:
    MODEL_PATH='./models/model_weights_{}'.format(wandb_run)
    if not os.path.exists('./models'):
        os.mkdir('./models')
    torch.save(model.state_dict(), MODEL_PATH)

In [ ]:
'''
Perform evaluation
'''
train_eval_dict = model.evaluate_batch(X_train.to(device), Y_train.to(device))
test_eval_dict = model.evaluate_batch(X_test.to(device), Y_test.to(device))

In [ ]:
X_test.shape

In [ ]:
X_train.shape

In [ ]:
Y_train.shape

In [ ]:
Y_test.shape

## 4. Plotting and Evaluation

In [ ]:
'''
Create plot tables for T+n th predictions
'''
train_gt = train_eval_dict['y_true']
train_gt_df = pd.DataFrame(train_gt.cpu().numpy()[:,:,0])
train_gt_values = np.append(train_gt_df[0].values, train_gt_df.iloc[-1,1:]) # ground-truth values for train data

test_gt = test_eval_dict['y_true']
test_gt_df = pd.DataFrame(test_gt.cpu().numpy()[:,:,0])
test_gt_values = np.append(test_gt_df[0].values, test_gt_df.iloc[-1,1:]) # ground-truth values for test data

train_pred = train_eval_dict['y_pred'] # model predicted values for train data
test_pred = test_eval_dict['y_pred'] # model predicted values for test data

train_T_pred_table, train_plot_df, plot_train_gt_values = utils.predictionTable(df_train, train_pred, train_gt_values)

test_T_pred_table, test_plot_df, plot_test_gt_values = utils.predictionTable(df_test, test_pred, test_gt_values)

In [ ]:
'''
Generate the plots on train data
'''

# Specify the list of T+n predictions to plot
horizon_range = [1,7, 14] # this will plot T+1 and T+n predictions w.r.t Ground truth

utils.plotTable(train_plot_df, plot_train_gt_values, horizon_range)

In [ ]:
'''
Generate the plots on test data
'''

# Specify the list of T+n predictions to plot
horizon_range = [1,7,14] # this will plot T+1 and T+n predictions w.r.t Ground truth

utils.plotTable(test_plot_df, plot_test_gt_values, horizon_range)

#### Compute the RMSE values

In [ ]:
'''
Compute train rmse

- Train RMSE values for all T+n th predictions. The index represents the T+n

'''
rmse_values = []
for i in range(output_window):
    rmse_values.append(utils.compute_rmse(i, train_T_pred_table, train_gt_values))
rmse_values = pd.DataFrame(rmse_values, columns=['RMSE'], index=range(1,output_window+1))
rmse_values

In [ ]:
'''
Compute test rmse

- Test RMSE values for all T+n th predictions. The index represents the T+n

'''
test_rmse_values = []
for i in range(output_window):
    test_rmse_values.append(utils.compute_rmse(i, test_T_pred_table, test_gt_values))
test_rmse_values = pd.DataFrame(test_rmse_values, columns=['RMSE'], index=range(1,output_window+1))
test_rmse_values